In [4]:
import os
import pandas as pd

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)

dir_path = "xq_import"
files = os.listdir(dir_path)
sorted_files = sorted(files)
last_file = sorted_files[-1]

# 輸出最後一個檔案的名稱
print(last_file)
with open(f"xq_import\{last_file}" ,'r', encoding='utf-8') as file:
    contents = file.read()
# print(contents)

df_stockName = pd.read_csv("paras/股票名稱.csv")
df_stockName.columns = ["stockId","stockName"]
df_stockName.index = df_stockName["stockId"]
df_stockName["stockId"] = df_stockName["stockId"].astype("string")
# print(df_stockName)

import finlab
finlab.login(open("config.txt", "r").read())
from finlab import data
import pandas as pd
import _beowFmt as fm 
import time

##
closes   = data.get("price:收盤價").fillna(0).astype(float)[240:]
opens    = data.get("price:開盤價").fillna(0).astype(float)[240:]      #如果資料有空白自動填0
highs    = data.get("price:最高價").fillna(0).astype(float)[240:]
lows     = data.get("price:最低價").fillna(0).astype(float).tail(1)

closeEma5   = closes.rolling(5, min_periods=1).mean().round(3).astype(float).tail(1)
closeEma13  = closes.rolling(13, min_periods=1).mean().round(3).astype(float).tail(1)
closeEma34  = closes.rolling(34, min_periods=1).mean().round(3).astype(float).tail(1)
volumns     = data.get("price:成交股數").fillna(0).astype(float)[250: ]
v           = (volumns/1000).round(0).astype(int)
ylows       = lows.shift(1, axis = 0)
ycloses     = closes.shift(1, axis = 0) 

vdf = v.iloc[-1:].transpose()
vdf["id"] = vdf.index
vdf.columns = ["ma1", "id"]
vdf["id"] = vdf["id"].astype("string")
# print(vdf.info())
# vdf = pd.merge(vdf, df_stockName, left_on = "id", right_on = "stockId" , validate='one_to_one')
vdf["open"] = opens.iloc[-1:].transpose()
vdf["close"] = closes.iloc[-1:].transpose()
vdf["high"] = highs.iloc[-1:].transpose()
vdf["closeEma5"] = closeEma5.iloc[-1:].transpose()
vdf["closeEma13"] = closeEma13.iloc[-1:].transpose()
vdf["closeEma34"] = closeEma34.iloc[-1:].transpose()
vdf["low"] = lows.iloc[-1:].transpose()
# vdf["ylow"] = ylows.iloc[-1:].transpose()
vdf["yclose"] = ycloses.iloc[-1:].transpose()
vdf["Rbar"] = vdf["close"] >= vdf["open"]
vdf["EE"] = (vdf["high"] >= vdf["closeEma5"]) & (vdf["closeEma5"] >= vdf["closeEma13"]) & (vdf["closeEma13"] >= vdf["closeEma34"]) & (vdf["high"] >= vdf["closeEma13"]) & (vdf["closeEma13"] >= vdf["open"]) & (vdf["closeEma5"] >= vdf["open"]) & (vdf["close"] >= vdf["closeEma34"])
# print(vdf)

vdf2 = vdf[ (vdf["EE"] ==True) & (vdf["open"] >= 10) ].dropna()
s1 ,s2 , o_nowTime = '', '', time.strftime("%m%d_1400", time.localtime())

# print(vdf2)

dd = vdf2["id"].tolist()
for d in dd:
  s1 += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態2023{o_nowTime}_挑戰Ema5_13.csv", s1)

量價型態20230910_1400_挑戰Ema5_13.csv
輸入成功!
